In [1]:
import logging
# set seed
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
import utils as ut
import experiment as exp
from evaluation import *
from sklearn.metrics import mean_squared_error
from torch.utils.tensorboard import SummaryWriter
import torch
import random
#define fixed_hyperparams and create a config gen
from configurations import RandomConfigGen, Configuration
from torch import nn
from deep_net import RandomNet
from experiment import run_experiment
import regex as re
from pathlib import *
from plot import *
from sk_models import setup_pls_models_exh, StandardScaler, PLSRegression, DeepKNN,CustomWrapper
from tqdm.notebook import tqdm, trange

seed = 1
torch.manual_seed(seed)
random.seed(seed + 1)
np.random.seed(seed + 2)
random_state = np.random.RandomState(seed)
import gc
torch.cuda.empty_cache()
gc.collect()

print(f"GPU detected is {torch.cuda.get_device_name(0)}")

GPU detected is GeForce GTX 970


In [2]:
id_col_db = {'A_C_OF_ALPHA':["sample_id"],
             'A_C_OF_SIWARE':[],
             'A_AL_RT':[],
             'PLN7':["db_id", "sample_id"],
             'mango_684_990': ['Set','Season','Region','Date','Type','Cultivar','Pop','Temp',"FruitID"]
            }

output_col_db= {'A_C_OF_ALPHA':None,
             'A_C_OF_SIWARE':None,
             'A_AL_RT':None,
             'PLN7':None,
             'mango_684_990': ['DM']
            }


In [3]:
#setup input and output formats, load data

file_name = "A_C_OF_ALPHA.csv"
dataset_name = re.sub(r'\.(?=csv$)[^.]+$', '',file_name)


data_path = Path('D:/workspace/lazydeep/data/soil_data/')
model_path = Path('D:/workspace/lazydeep/experiments/2.00/')
log_path = Path("D:/workspace/lazydeep/experiments/2.02")

data_file = data_path / file_name
log_dir = log_path / re.sub(r'\.(?=csv$)[^.]+$', '',file_name)
model_dir = model_path / re.sub(r'\.(?=csv$)[^.]+$', '',file_name)

if not log_dir.exists():
    log_dir.mkdir()
print(log_dir)

id_cols =id_col_db[dataset_name]
output_cols = output_col_db[dataset_name]


D:\workspace\lazydeep\experiments\2.02\A_C_OF_ALPHA


In [4]:
data = pd.read_csv(data_file)
data = data.sample(frac=1)
data = ut.sample_data(data,random_state)
nrow, ncol = data.shape
n_features = ncol - 1-len(id_cols)

dataset = TabularDataset(data,id_cols = id_cols, cat_cols=None, output_cols=output_cols, ignore_cols= None)
print(data.shape)

(7329, 1703)


In [5]:
# set logging, in this case the root logger
ut.setup_logger(logger_name="",file_name=log_dir/"log.txt")
ut.setup_logger(logger_name="summary",file_name=log_dir/"summary.txt")
summary_logger = logging.getLogger("summary")
tb = SummaryWriter(log_dir/"tb")


In [6]:
if False: 
    n_models = 100
    model_names = [f"random_{i}" for i in range(0,n_models)]
    deep_models = {name:torch.load(model_dir/"models"/name/"_model") for name in model_names}
    configs =  {name:Configuration().load(model_dir/"models"/name/"_config") for name in model_names}
    #for each model, load state
    print(f"Loaded {len(deep_models)} models")
    #print(deep_models)
            
n_models = 100
epochs = 100
bs = 32
fixed_hyperparams = {'bs': bs,'loss': nn.MSELoss(),'epochs': epochs}
device = "cpu"#torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#setup models
config_gen = RandomConfigGen(lr= (0,1),
                             allow_increase_size=False,
                             n_features=50,
                             opt=[torch.optim.SGD,
                                  torch.optim.Adam],
                             lr_update = [None,
                                          torch.optim.lr_scheduler.ReduceLROnPlateau,
                                          torch.optim.lr_scheduler.ExponentialLR,
                                          torch.optim.lr_scheduler.CosineAnnealingLR],
                            dropout = [True,False],
                            batch_norm = [True,False])
configs = {f"random_{i}":config_gen.sample() for i in range(n_models)}
config_gen.save(log_dir/'config_gen.txt')

models = {name:RandomNet(input_size=n_features,
                         n_layers=config.n_layers,
                         act_function=config.act_function,
                         n_features = config.n_features,
                         dropout=config.dropout,
                         batch_norm=config.batch_norm,
                         device=device,dtype=torch.float)
          for name, config in configs.items()}
preprocessing = StandardScaler()

model_names = models.keys()
for name in model_names:
        sub_path = log_dir / name
        if not sub_path.exists():
            sub_path.mkdir()

In [7]:
if dataset_name == 'mango_684_990':
    eval_ = MangoesSplitter(preprocessing=preprocessing,tensorboard=None,time=True,random_state=random_state)
else:
    eval_ = CrossValEvaluation(preprocessing=preprocessing,tensorboard=None,time=True,random_state=random_state)

In [8]:
scores={} #->model->knn:{fold_0:number,...,fold_n:number,mean:number,median:number
preds={} #model-> foldsxknn_models
deep_scores_dict={}
deep_preds_dict={}
actual_y = None

load_fun_cv = lambda name,model, fold : model.load_state(model_dir/'models'/name/f"_fold_{fold}")
load_fun_pp_cv = None #lambda fold : preprocessing.from_state(preprocessing.load_state(model_dir/'preprocessing'/f"_fold_{fold}"))
load_fun_build = lambda name,model : model.load_state(model_dir/'models'/name/f"_final")
load_fun_pp_build = None #lambda : preprocessing.from_state(preprocessing.load_state(model_dir/'preprocessing'/f"_final"))


In [9]:
deep_scheme = DeepScheme(configs, fixed_hyperparams=fixed_hyperparams,loss_eval=loss_target,device=device,tensorboard=tb,adaptive_lr=False,update=False)
deep_scores, deep_preds, _ , _, _,_ = eval_.evaluate(models,dataset,deep_scheme,logger_name="log",load_fun=load_fun_cv,load_fun_pp=load_fun_pp_cv)
deep_scores_final, deep_preds_final, _ ,_, _,_ = eval_.build(models,dataset,deep_scheme,logger_name="test_log",load_fun=load_fun_build,load_fun_pp=load_fun_pp_build)

Running Cross Evaluation with 5 folds'
-----------------------------------Fold 0 - Train 3663 - Val 1222 - Test 1222-----------------------------------'
Tested (test) on 1222 instances with mean losses of: random_0:12.3517,random_1:10.4208,random_2:603.8857,random_3:11.1601,random_4:12.3001,random_5:8.3633,random_6:607.3073,random_7:607.7903,random_8:604.4302,random_9:8.9701,random_10:10.3307,random_11:11.0018,random_12:11.4681,random_13:16.1366,random_14:108.9007,random_15:9.9472,random_16:15.5362,random_17:16.5022,random_18:14.4068,random_19:9.6157,random_20:15.0358,random_21:38.3918,random_22:94.0412,random_23:19.2689,random_24:13.7357,random_25:12.0343,random_26:11.434,random_27:11.8311,random_28:16.3709,random_29:9.5527,random_30:24.8279,random_31:11.6792,random_32:611.1023,random_33:599.2791,random_34:19.1817,random_35:12.6499,random_36:11.8897,random_37:9.9449,random_38:11.5293,random_39:604.3834,random_40:15.1597,random_41:8.9979,random_42:13.3156,random_43:606.8097,random_44:9

In [10]:
all_scores = []
for k,v in ut.flip_dicts(deep_scores).items():
    dict1 = {'model_num':k,"predictor":"deep"}
    all_scores.append({**dict1,**v})

all_scores_final = []
for k,v in ut.flip_dicts(deep_scores_final).items():
    dict1 = {'model_num':k,"predictor":"deep"}
    all_scores_final.append({**dict1,**v})

In [11]:
scores_df_sorted = pd.DataFrame(all_scores).sort_values(by='MSE')
for i,(index,row) in enumerate(scores_df_sorted.iterrows()):
    s = f"{i} - " + " - ".join([f"{i}" for i in row.tolist()])
    print(s)

0 - random_80 - deep - 9.35667500285197 - 8.285823323722752 - 6.716800460456142 - 8.608006351995819 - 15.8545239938272 - 9.764056871816873 - 0.9585970043533256
1 - random_5 - deep - 8.363282239573676 - 11.932893898990462 - 6.45366010267553 - 12.08134427146771 - 15.690954698879732 - 10.904179259704094 - 0.9537624890609554
2 - random_77 - deep - 10.129997791908377 - 10.986665495093638 - 6.516540925684374 - 10.833087069006067 - 16.286100628139735 - 10.95034995834599 - 0.9535667092473011
3 - random_10 - deep - 10.33067052610182 - 11.382877679191127 - 7.954366040366483 - 9.834830644577863 - 16.03341594980351 - 11.107150273958244 - 0.9529018213969005
4 - random_82 - deep - 9.545721391218969 - 10.144600848714576 - 7.3080058531327685 - 11.571789434168986 - 17.847468303521083 - 11.283046063033243 - 0.9521559621004065
5 - random_79 - deep - 10.980880799738358 - 8.982656263485673 - 7.754505259008509 - 10.597175655942975 - 19.866092637364343 - 11.635720435904203 - 0.9506605001508949
6 - random_44 

In [12]:
scores_df_sorted_final = pd.DataFrame(all_scores_final).sort_values(by='MSE')

for i,(index,row) in enumerate(scores_df_sorted_final.iterrows()):
    s = f"{i} - " + " - ".join([f"{i}" for i in row.tolist()])
    print(s)

0 - random_47 - deep - 8.919891087940544 - 0.9624491752872761
1 - random_74 - deep - 9.330148315122123 - 0.9607220805197345
2 - random_9 - deep - 9.365010362338474 - 0.96057531879235
3 - random_67 - deep - 9.372335621272377 - 0.9605444810263403
4 - random_77 - deep - 10.410104090572132 - 0.9561756987734098
5 - random_54 - deep - 10.539051099389242 - 0.9556328595753093
6 - random_80 - deep - 10.644197250402254 - 0.9551902168740721
7 - random_34 - deep - 10.644277164193543 - 0.9551898804541833
8 - random_76 - deep - 11.192283060293786 - 0.9528828933908748
9 - random_93 - deep - 11.209716252752152 - 0.9528095033967893
10 - random_35 - deep - 11.381222868815062 - 0.952087497397685
11 - random_11 - deep - 11.45432152691222 - 0.9517797677550379
12 - random_94 - deep - 11.475549558651355 - 0.9516904023030416
13 - random_22 - deep - 11.479450694018842 - 0.9516739793614473
14 - random_90 - deep - 11.556877536235833 - 0.9513480289937257
15 - random_92 - deep - 11.698672771713959 - 0.950751101522

In [13]:
def build_predictors(n):
    predictors = {}
    for i in [5,10,20,50,100,200,500,1000]:
        if i* 2 < n:
            predictors[f'knn_unif_n={i}'] = CustomWrapper(KNeighborsRegressor(n_neighbors=i, weights='uniform'))
            predictors[f'knn_dist_n={i}'] = CustomWrapper(KNeighborsRegressor(n_neighbors=i, weights='distance'))
    return predictors


for deep_name,deep_model in tqdm(models.items()):
    logging.getLogger().info(f"Running model {deep_name}")
    temp_dict = {deep_name:deep_model}

    lwr_scheme = DeepLWRScheme_1_to_n(lwr_models = build_predictors(nrow),n_neighbours=500,loss_fun_sk = mean_squared_error)
    lwr_scores, lwr_preds, _ , _, _,_= eval_.evaluate(temp_dict,dataset,lwr_scheme,logger_name="log")
    lwr_scores_final, lwr_preds_final, _ , _, _,_= eval_.build(temp_dict,dataset,lwr_scheme,logger_name="test_log")

    #scores
    for k,v in ut.flip_dicts(lwr_scores).items(): 
        dict1 = {'model_num':deep_name,"predictor":k}
        all_scores.append({**dict1,**v})

    for k,v in ut.flip_dicts(lwr_scores_final).items():
        dict1 = {'model_num':deep_name,"predictor":k}
        all_scores_final.append({**dict1,**v})

    lwr_preds['deep'] = deep_preds[deep_name]
    lwr_preds_final['deep'] = deep_preds_final[deep_name]

    lwr_preds.to_csv(log_dir/deep_name/ f"predictions.csv",index=False)
    lwr_preds_final.to_csv(log_dir/deep_name/ f"predictions_test.csv",index=False)

    #preds
    # todo save predictions - appending solns
    plot_preds_and_res(lwr_preds,name_lambda=lambda x:f"{deep_name} with {x} predictor",save_lambda= lambda x:f"deep_lwr{x}",save_loc=log_dir/deep_name)


  0%|          | 0/100 [00:00<?, ?it/s]

Running model random_0'
Running Cross Evaluation with 5 folds'
-----------------------------------Fold 0 - Train 3663 - Val 1222 - Test 1222-----------------------------------'
Finished training DeepLWR with a train loss of knn_unif_n=5:8.4988,knn_dist_n=5:0.0,knn_unif_n=10:10.3282,knn_dist_n=10:0.0,knn_unif_n=20:12.8968,knn_dist_n=20:0.0,knn_unif_n=50:20.0656,knn_dist_n=50:0.0,knn_unif_n=100:29.8583,knn_dist_n=100:0.0,knn_unif_n=200:42.7681,knn_dist_n=200:0.0,knn_unif_n=500:62.4546,knn_dist_n=500:0.0,knn_unif_n=1000:81.4848,knn_dist_n=1000:0.0'
Tested (test) on 1222 instances with mean losses of: knn_unif_n=5:14.4657,knn_dist_n=5:14.812,knn_unif_n=10:13.9371,knn_dist_n=10:14.1192,knn_unif_n=20:15.0256,knn_dist_n=20:13.9255,knn_unif_n=50:24.8886,knn_dist_n=50:16.0742,knn_unif_n=100:41.9565,knn_dist_n=100:21.2799,knn_unif_n=200:64.2314,knn_dist_n=200:31.6769,knn_unif_n=500:95.4394,knn_dist_n=500:55.5767,knn_unif_n=1000:122.77,knn_dist_n=1000:81.4597'
-----------------------------------F

C:\Users\huonf\.conda\envs\lazydeep\lib\site-packages\scipy\stats\_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


Running model random_1'
Running Cross Evaluation with 5 folds'
-----------------------------------Fold 0 - Train 3663 - Val 1222 - Test 1222-----------------------------------'
Finished training DeepLWR with a train loss of knn_unif_n=5:6.6434,knn_dist_n=5:0.0,knn_unif_n=10:8.1904,knn_dist_n=10:0.0,knn_unif_n=20:10.3728,knn_dist_n=20:0.0,knn_unif_n=50:17.3424,knn_dist_n=50:0.0,knn_unif_n=100:26.5811,knn_dist_n=100:0.0,knn_unif_n=200:40.099,knn_dist_n=200:0.0,knn_unif_n=500:60.0502,knn_dist_n=500:0.0,knn_unif_n=1000:79.6363,knn_dist_n=1000:0.0'
Tested (test) on 1222 instances with mean losses of: knn_unif_n=5:12.3692,knn_dist_n=5:11.8578,knn_unif_n=10:11.306,knn_dist_n=10:11.1362,knn_unif_n=20:12.4035,knn_dist_n=20:10.8118,knn_unif_n=50:23.0564,knn_dist_n=50:12.5137,knn_unif_n=100:39.0177,knn_dist_n=100:17.3839,knn_unif_n=200:61.7055,knn_dist_n=200:28.1326,knn_unif_n=500:93.1856,knn_dist_n=500:52.8405,knn_unif_n=1000:120.8271,knn_dist_n=1000:78.6592'
-----------------------------------F

C:\Users\huonf\.conda\envs\lazydeep\lib\site-packages\scipy\stats\_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


...

C:\Users\huonf\.conda\envs\lazydeep\lib\site-packages\scipy\stats\_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


Running model random_98'
Running Cross Evaluation with 5 folds'
-----------------------------------Fold 0 - Train 3663 - Val 1222 - Test 1222-----------------------------------'
Finished training DeepLWR with a train loss of knn_unif_n=5:4.946,knn_dist_n=5:0.0,knn_unif_n=10:6.4297,knn_dist_n=10:0.0,knn_unif_n=20:9.4006,knn_dist_n=20:0.0,knn_unif_n=50:16.9378,knn_dist_n=50:0.0,knn_unif_n=100:27.5378,knn_dist_n=100:0.0,knn_unif_n=200:42.0552,knn_dist_n=200:0.0,knn_unif_n=500:63.9153,knn_dist_n=500:0.0,knn_unif_n=1000:85.5017,knn_dist_n=1000:0.0'
Tested (test) on 1222 instances with mean losses of: knn_unif_n=5:8.907,knn_dist_n=5:8.665,knn_unif_n=10:8.975,knn_dist_n=10:8.5385,knn_unif_n=20:10.451,knn_dist_n=20:8.9026,knn_unif_n=50:22.065,knn_dist_n=50:13.5854,knn_unif_n=100:39.9804,knn_dist_n=100:23.017,knn_unif_n=200:64.0985,knn_dist_n=200:40.1885,knn_unif_n=500:97.4391,knn_dist_n=500:72.8707,knn_unif_n=1000:127.0904,knn_dist_n=1000:104.0322'
-----------------------------------Fold 1 - T

C:\Users\huonf\.conda\envs\lazydeep\lib\site-packages\scipy\stats\_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


Running model random_99'
Running Cross Evaluation with 5 folds'
-----------------------------------Fold 0 - Train 3663 - Val 1222 - Test 1222-----------------------------------'
Finished training DeepLWR with a train loss of knn_unif_n=5:5.5571,knn_dist_n=5:0.0,knn_unif_n=10:6.9135,knn_dist_n=10:0.0,knn_unif_n=20:9.4283,knn_dist_n=20:0.0,knn_unif_n=50:17.0436,knn_dist_n=50:0.0,knn_unif_n=100:27.68,knn_dist_n=100:0.0,knn_unif_n=200:42.0496,knn_dist_n=200:0.0,knn_unif_n=500:64.5513,knn_dist_n=500:0.0,knn_unif_n=1000:85.7356,knn_dist_n=1000:0.0'
Tested (test) on 1222 instances with mean losses of: knn_unif_n=5:9.8132,knn_dist_n=5:9.5797,knn_unif_n=10:8.6068,knn_dist_n=10:8.5521,knn_unif_n=20:10.9162,knn_dist_n=20:9.614,knn_unif_n=50:21.1653,knn_dist_n=50:15.1032,knn_unif_n=100:39.3077,knn_dist_n=100:26.1643,knn_unif_n=200:63.685,knn_dist_n=200:44.4691,knn_unif_n=500:98.3223,knn_dist_n=500:77.0166,knn_unif_n=1000:127.6618,knn_dist_n=1000:106.7282'
-----------------------------------Fold 1 

C:\Users\huonf\.conda\envs\lazydeep\lib\site-packages\scipy\stats\_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [ ]:
scores_df = pd.DataFrame(all_scores)
scores_df.to_csv(log_dir/f"scores.csv",index=False)

scores_df_sorted = pd.DataFrame(scores_df).sort_values(by='MSE')

best_5 = []
summary_logger.info(f"Rank - " +" - ".join(list(scores_df_sorted.columns)))
for i,(index,row) in enumerate(scores_df_sorted.iterrows()):
    if i < 5:
        best_5.append((row["model_num"],row["predictor"],row["MSE"],row["R2"]))
    s = f"{i} - " + " - ".join([f"{i}" for i in row.tolist()])
    summary_logger.info(s)

Rank - model_num - predictor - fold_0 - fold_1 - fold_2 - fold_3 - fold_4 - MSE - R2'
0 - random_47 - knn_dist_n=5 - 4.595033800415356 - 4.752693827309585 - 3.613904959449901 - 7.194256687415775 - 24.70315088835913 - 8.970400485768685 - 0.9619623832668426'
1 - random_47 - knn_unif_n=5 - 4.9756313670277015 - 4.77119461373253 - 3.649414813289418 - 7.168352093101532 - 25.56390707018252 - 9.224274647762439 - 0.9608858685574169'
2 - random_67 - knn_dist_n=5 - 4.67120851643114 - 4.8700470794737925 - 3.7240975649153403 - 8.563662744653598 - 25.12510977961715 - 9.389312054542664 - 0.9601860525753211'
3 - random_66 - knn_dist_n=5 - 6.291564429360594 - 5.228751808383108 - 4.608749975051153 - 6.290412125402438 - 25.320405718111143 - 9.546736327000222 - 0.9595185189295561'
4 - random_67 - knn_unif_n=5 - 5.041888569115387 - 4.995189675642185 - 3.7959639605417697 - 9.016605520247586 - 25.793725358900904 - 9.727132080294016 - 0.9587535782187189'
5 - random_80 - deep - 9.35667500285197 - 8.28582332372

In [ ]:
scores_df_final = pd.DataFrame(all_scores_final)
scores_df_final.to_csv(log_dir/f"test_scores.csv",index=False)

summary_logger.info("-----------------------\n Best 5 on Test Sest \n ---------------------")
summary_logger.info(f"Rank -  Deep Model - Predictor - Val Set - Test Set")
for i, (j,k,v,x) in enumerate(best_5):
    row = scores_df_final.loc[(scores_df_final['model_num']==j) & (scores_df_final['predictor'] == k)].iloc[0]
    #print(row)
    s = f"{i} - {j} - {k} - {v} - {x} - {row['MSE']} - {row['R2']}"
    summary_logger.info(s)

In [ ]:
#graph our deep models by rank - plot - then overlay our knn moels

deep_set = scores_df[scores_df["predictor"]=="deep"].sort_values("R2")
deep_set["order"] = [i for i in range(0,100)]
deep_ordering = {row["model_num"]:row["order"] for index, row in deep_set.iterrows()}

def order_models(x):
    x = [deep_ordering[i] for i in x]
    return x

fig, ax = plt.subplots()
set_deep = False
knn_models = scores_df["predictor"].unique()
for knn_model in knn_models:
    subset = scores_df[scores_df["predictor"]==knn_model]
    s=3
    if knn_model == "deep":
        s=10
    ax.scatter(x=order_models(subset["model_num"].tolist()), y=subset["R2"], s=s, label=knn_model)

#ax.set_ylim(0,scores_db["deep_mean"].max())
ax.set_ylim(0,1)
# plot residuals
ax.legend(loc='upper right',bbox_to_anchor=(1.4, 1))
ax.set_ylabel("R^2 Score")
ax.set_xlabel("Deep Model Rank")
#ax.set_ylim(0,200)
#ax.set_yscale("symlog")
ax.set_title("Summary of LWR improvements over Deep Models")
plt.savefig(log_dir/f"summary_plot.png", bbox_inches='tight')

In [ ]:
scores_df_base = scores_df[scores_df["predictor"]=='deep']
scores_df_dist = scores_df[scores_df["predictor"].str.contains('dist')]   #val_eq_list(scores_df["predictor"],'dist')] #np.logical_or(scores_df["predictor"]=="deep",'dist' in scores_df["predictor"])]
scores_df_unif = scores_df[scores_df["predictor"].str.contains('unif')] 

In [ ]:
fig, ax = plt.subplots()
knn_models = scores_df_dist["predictor"].unique()
ax.scatter(x=order_models(scores_df_base["model_num"].tolist()), y=scores_df_base["R2"], s=10, label='deep')
for knn_model in knn_models:
    subset = scores_df_dist[scores_df_dist["predictor"]==knn_model]
    s=3
    ax.scatter(x=order_models(subset["model_num"].tolist()), y=subset["R2"], s=s, label=knn_model)




#ax.set_ylim(0,scores_db["deep_mean"].max())
ax.set_ylim(0,1)
# plot residuals
ax.legend(loc='upper right',bbox_to_anchor=(1.4, 1))
ax.set_ylabel("R^2 Score")
ax.set_xlabel("Deep Model Rank")
#ax.set_ylim(0,200)
#ax.set_yscale("symlog")
ax.set_title("Summary of LWR improvements over Deep Models")
plt.savefig(log_dir/f"summary_plot_dist.png", bbox_inches='tight')
logging.getLogger().info("Wrote Summary Graph")

fig, ax = plt.subplots()
knn_models = scores_df_dist["predictor"].unique()
    

for knn_model in knn_models:
    subset = scores_df_dist[scores_df_dist["predictor"]==knn_model]
    s=3
    y1 = subset["R2"].to_numpy() - scores_df_base["R2"].to_numpy()
    ax.scatter(x=order_models(subset["model_num"].tolist()), y=y1, s=s, label=knn_model)

ax.set_ylim(-1,1)

ax.legend(loc='upper right',bbox_to_anchor=(1.4, 1))
ax.set_ylabel("Difference, KNN R2 - BaseR2")
ax.set_xlabel("Deep Model Rank")

ax.set_title("Summary of LWR improvements over Deep Models")
plt.savefig(log_dir/f"improve_plot_dist.png", bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots()
set_deep = False
knn_models = scores_df_unif["predictor"].unique()
ax.scatter(x=order_models(scores_df_base["model_num"].tolist()), y=scores_df_base["R2"], s=10, label='deep')
for knn_model in knn_models:
    subset = scores_df_unif[scores_df_unif["predictor"]==knn_model]
    s=3
    ax.scatter(x=order_models(subset["model_num"].tolist()), y=subset["R2"], s=s, label=knn_model)




#ax.set_ylim(0,scores_db["deep_mean"].max())
ax.set_ylim(0,1)
# plot residuals
ax.legend(loc='upper right',bbox_to_anchor=(1.4, 1))
ax.set_ylabel("R^2 Score")
ax.set_xlabel("Deep Model Rank")
#ax.set_ylim(0,200)
#ax.set_yscale("symlog")
ax.set_title("Summary of LWR improvements over Deep Models")
plt.savefig(log_dir/f"summary_plot_unif.png", bbox_inches='tight')
logging.getLogger().info("Wrote Summary Graph")

fig, ax = plt.subplots()
knn_models = scores_df_unif["predictor"].unique()
for knn_model in knn_models:
    subset = scores_df_unif[scores_df_unif["predictor"]==knn_model]
    s=3
    y1 = subset["R2"].to_numpy() - scores_df_base["R2"].to_numpy()
    
    ax.scatter(x=order_models(subset["model_num"].tolist()), y=y1, s=s, label=knn_model)

ax.set_ylim(-1,1)

ax.legend(loc='upper right',bbox_to_anchor=(1.4, 1))
ax.set_ylabel("Difference, KNN R2 - BaseR2")
ax.set_xlabel("Deep Model Rank")

ax.set_title("Summary of LWR improvements over Deep Models")
plt.savefig(log_dir/f"improve_plot_unif.png", bbox_inches='tight')


In [ ]:
import seaborn as sns
from matplotlib.colors import Colormap
class nlcmap(object):
    def __init__(self, cmap, levels):
        self.cmap = cmap
        self.N = cmap.N
        self.monochrome = self.cmap.monochrome
        self.levels = np.asarray(levels, dtype='float64')
        self._x = self.levels
        self.levmax = self.levels.max()
        self.levmin = self.levels.min()
        self.transformed_levels = np.linspace(self.levmin, self.levmax, #uniform spacing along levels (colour segments)
             len(self.levels))

    def __call__(self, xi, alpha=1.0, **kw):
        yi = np.interp(xi, self._x, self.transformed_levels)
        return self.cmap((yi-self.levmin) / (self.levmax-self.levmin), alpha)
    
levels = np.concatenate((
    [0, 1],
    [0.6,0.8,0.9,0.95,0.98]
    ))

levels = levels[levels <= 1]
levels.sort()
print(levels)
cmap_nonlin = nlcmap(plt.cm.YlGnBu, levels) 

In [ ]:
subset = scores_df_base[["model_num",'predictor' ,"R2"]]
#subset = subset.sort_values('model_num', key=order_models)
#trans = subset["predictor"].transform(lambda x: (x.replace("knn_dist_n=",""))).tolist()
#subset.loc[:,"predictor"]=trans

#trans = subset["model_num"].transform(lambda x: int(x.replace("random_",""))).tolist()
#subset.loc[:,"model_num"]=trans

#subset=subset.sort_values("model_num",ascending=False)
wide = subset.pivot(index = "predictor",columns= "model_num",values="R2")
wide = wide.sort_index(axis=1,key=order_models)

ax = sns.heatmap(wide, linewidth=0.0,vmin=0,center=0,cbar_kws={'label':"R2 Score"},cmap=cmap_nonlin)

ax.set_title("Grid Search for number of neighbours and deep model ")
ax.set_xlabel("Deep Model")
ax.set_ylabel("Number of Neighbours")

plt.savefig(log_dir/"pls_heatmap.png", bbox_inches='tight')


In [ ]:
# heat map for distance based knn and deep model

subset = scores_df_dist[["model_num","predictor","R2"]]
subset = subset.sort_values('model_num', key=order_models)

trans = subset["predictor"].transform(lambda x: int(x.replace("knn_dist_n=",""))).tolist()
subset.loc[:,"predictor"]=trans

#trans = subset["model_num"].transform(lambda x: int(x.replace("random_",""))).tolist()
#subset.loc[:,"model_num"]=trans

#subset=subset.sort_values("model_num",ascending=False)
wide = subset.pivot(index = "predictor",columns= "model_num",values="R2")
wide = wide.sort_index(axis=1,key=order_models)

ax = sns.heatmap(wide, linewidth=0.0,vmin=0,center=0,cbar_kws={'label':"R2 Score"},cmap=cmap_nonlin)

ax.set_title("Grid Search for number of neighbours and deep model ")
ax.set_xlabel("Deep Model")
ax.set_ylabel("Number of Neighbours")

#plt.savefig(log_dir/"pls_heatmap.png", bbox_inches='tight')

In [ ]:
# heat map for uniform  based knn and deep model

subset = scores_df_unif[["model_num","predictor","R2"]]
subset = subset.sort_values('model_num', key=order_models)

trans = subset["predictor"].transform(lambda x: int(x.replace("knn_unif_n=",""))).tolist()
subset.loc[:,"predictor"]=trans

#trans = subset["model_num"].transform(lambda x: int(x.replace("random_",""))).tolist()
#subset.loc[:,"model_num"]=trans

#subset=subset.sort_values("model_num",ascending=False)
wide = subset.pivot(index = "predictor",columns= "model_num",values="R2")
wide = wide.sort_index(axis=1,key=order_models)

ax = sns.heatmap(wide, linewidth=0.0,vmin=0,center=0,cbar_kws={'label':"R2 Score"},cmap=cmap_nonlin)

ax.set_title("Grid Search for number of neighbours and deep model ")
ax.set_xlabel("Deep Model")
ax.set_ylabel("Number of Neighbours")

#plt.savefig(log_dir/"pls_heatmap.png", bbox_inches='tight')


In [ ]:
#graph our deep models by rank on final set - plot - then overlay our knn moels

deep_set = scores_df_final[scores_df_final["predictor"]=="deep"].sort_values("R2")
deep_set["order"] = [i for i in range(0,100)]
deep_ordering = {row["model_num"]:row["order"] for index, row in deep_set.iterrows()}

def order_models(x):
    x = [deep_ordering[i] for i in x]
    return x

fig, ax = plt.subplots()
set_deep = False
knn_models = scores_df_final["predictor"].unique()
for knn_model in knn_models:
    subset = scores_df_final[scores_df_final["predictor"]==knn_model]
    s=3
    if knn_model == "deep":
        s=10
    ax.scatter(x=order_models(subset["model_num"].tolist()), y=subset["R2"], s=s, label=knn_model)

#ax.set_ylim(0,scores_db["deep_mean"].max())
ax.set_ylim(0,1)
# plot residuals
ax.legend(loc='upper right',bbox_to_anchor=(1.4, 1))
ax.set_ylabel("R^2 Score")
ax.set_xlabel("Deep Model Rank")
#ax.set_ylim(0,200)
#ax.set_yscale("symlog")
ax.set_title("Summary of LWR improvements over Deep Models")
plt.savefig(log_dir/f"summary_plot_final.png", bbox_inches='tight')

fig, ax = plt.subplots()
set_deep = False
knn_models = scores_df_final["predictor"].unique()
for knn_model in knn_models:
    subset = scores_df_final[scores_df_final["predictor"]==knn_model]
    s=3
    if knn_model == "deep":
        pass
    else:
        y1 = subset["R2"].to_numpy() - scores_df_final[scores_df_final["predictor"]=='deep']["R2"].to_numpy()
        ax.scatter(x=order_models(subset["model_num"].tolist()), y=y1, s=s, label=knn_model)

#ax.set_ylim(0,scores_db["deep_mean"].max())
ax.set_ylim(0,1)
# plot residuals
ax.legend(loc='upper right',bbox_to_anchor=(1.4, 1))
ax.set_ylabel("R^2 Score")
ax.set_xlabel("Deep Model Rank")
#ax.set_ylim(0,200)
#ax.set_yscale("symlog")
ax.set_title("Summary of LWR improvements over Deep Models")
plt.savefig(log_dir/f"improvement_plot_final.png", bbox_inches='tight')